In [1]:
import pandas as pd
import pymysql

# DB 연결
conn = pymysql.connect(
    host="localhost", # MariaDB 서버 주소
    port=3306,
    user="webuser",       # 사용자명
    password="webuser",  # 비밀번호
    database="DDProject",  # 사용할 데이터베이스
    charset="utf8mb4"  # 한글 지원
)



# transactional
try:
    conn.autocommit(False)
    cursor = conn.cursor()
    # 테이블 크리에이트
    create_table_sql = """CREATE TABLE IF NOT EXISTS subway (
        id INT AUTO_INCREMENT PRIMARY KEY,
        역사_ID INT,
        역사명 VARCHAR(50),
        호선 VARCHAR(50),
        위도 FLOAT,
        경도 FLOAT,
        지번주소 VARCHAR(255),
        지번거리 VARCHAR(20),
        도로명주소 VARCHAR(255),
        도로명거리 VARCHAR(20)
    )"""

    cursor.execute(create_table_sql)



    # 1. 데이터 로드
    df_subway = pd.read_csv('../result/서울시_전철역사정보.csv', encoding='utf-8', header=0)
    # 2. 데이터 변환

    # 3. 날짜 변환 (문자열 → DATETIME)

    # 테이블 인서트
    placeholders = ', '.join(['%s'] * df_subway.columns.values.size)
    insert_sql = f"""
        INSERT INTO subway (역사_ID,역사명,호선,위도,경도,지번주소,지번거리,도로명주소,도로명거리)
        VALUES ({placeholders})
    """

    data = df_subway[df_subway.columns.values].itertuples(index=False, name=None)
    cursor.executemany(insert_sql, data)
    # 변경 사항 저장
    conn.commit()

    #transactional error
except pymysql.MySQLError as e:
    print(e)
    conn.rollback()

finally:
    # 연결 종료
    cursor.close()
    conn.close()
# null_rows = df_subway[df_subway.isnull().any(axis=1)]
# null_rows

In [2]:
df_subway.dtypes

역사_ID           int64
역사명            object
호선             object
위도            float64
경도            float64
_PARCEL_AD     object
_PARCEL_DT     object
_ROAD_AD       object
_ROAD_DT       object
dtype: object